In [12]:
import sys
sys.path.append('..')
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time
import theano_lstm
import numpy as np

In [2]:
vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in xrange(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

[GpuElemwise{exp,no_inplace}(<CudaNdarrayType(float32, vector)>), HostFromGpu(GpuElemwise{exp,no_inplace}.0)]
Looping 1000 times took 0.463460 seconds
Result is [ 1.23178029  1.61879349  1.52278066 ...,  2.20771813  2.29967761
  1.62323296]
Used the gpu


In [27]:
num_examples = 35
input_seq = np.random.normal(size=(200, 300, num_examples)).astype('float32')
seq_lengths = [100 for x in range(num_examples)]
answers = np.zeros(shape=(50, num_examples)).astype('float32')
answers[10, :] = 1

In [10]:
%%time
reload(theano_lstm)
config.mode="FAST_COMPILE"
network = theano_lstm.lstm_rnn(300,
                  [(128, 128)],
                  50, log_dir='test_log')

In [28]:
%%time
_=network.adadelta_step(input_seq, seq_lengths, answers)

CPU times: user 752 ms, sys: 16 ms, total: 768 ms
Wall time: 768 ms


In [ ]:
%%time
reload(theano_lstm)
config.mode="FAST_RUN"
network_opt = theano_lstm.lstm_rnn(300,
                  [(128, 128)],
                  50, log_dir='test_log')